In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
from omnibus.data import plot, Field
from omnibus.formats.output import load
import omnibus.converters.meshtal as meshtal
import omnibus.converters.mctal as mctal
from cycler import cycler

/home/krowland/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
plot_dir = '..'
fwc_base_path = 'steel-plate-fwcadis/'
cad_base_path = 'steel-plate-cadis/'
mctal_path = '/output/mctal'
meshtal_path = '/output/meshtal'
outp_path = '/output/outp'

In [3]:
# plot_types = ["fwd", "fwc-adj","cad-adj", "mcnp"]
# for ptype in plot_types:
#     if not os.path.exists(plot_dir+"/"+ptype):
#         os.makedirs(plot_dir+"/"+ptype)

In [3]:
results = []
for quad_type in next(os.walk(fwc_base_path))[1]:
    for run in next(os.walk(fwc_base_path+quad_type))[1]:
        results.append({"type":quad_type,
                        "name":run,
                        "order": int(run[-2:])})
        
for run in results:
    if run["type"] == "ldo":
        run["angles"] = (run["order"]+1)*(run["order"]+1)
        run["label"]  = "LDO"
    elif run["type"] == "qr":
        run["angles"] = (run["order"])*(run["order"])*8
        run["label"]  = "QR"
    elif run["type"] == "galerkin":
        run["angles"] = run["order"]*(run["order"]+2)
        run["label"]  = "Galerkin"
    elif run["type"] == "ldfe":
        run["angles"] = 8*np.power(4,(run["order"]+1))
        run["label"]  = "LDFE"
    else:
        print "No angle formula for quadrature type " + run["type"]

In [4]:
def plot_outlines():
    # delineate f4 tally borders
    plt.plot([24,24], [130,135], 'k-')
    plt.plot([29,29], [130,135], 'k-')
    plt.plot([24,29], [135,135], 'k-')
    # plane separating source and steel
    plt.plot([0,53], [15,15], 'k-')
    # plane separating steel and water
    plt.plot([0,25], [30,30], 'k-')
    plt.plot([28,53], [30,30], 'k-')
    # plane separating steel and water
    plt.plot([0,53], [130,130], 'k-')
    # surfaces of steel plate
    plt.plot([25,25], [30,130], 'k-')
    plt.plot([28,28], [30,130], 'k-')

In [5]:
for run in results:
    if run["type"] == "ldo" and run["order"] == 11: # 144 angles
        ldo_dict = run
    if run["type"] == "qr" and run["order"] == 4: # 128 angles
        qr_dict = run
    if run["type"] == "galerkin" and run["order"] == 4: # P5 expansion
        gkn_dict = run
    if run["type"] == "ldfe" and run ["order"] == 1: # 128 angles
        ldfe_dict = run

MCNP CADIS plotting

In [8]:
for run in results:
    if run["type"] != "no-ww":
        mctal_file = cad_base_path+run["type"]+"/"+run["name"]+mctal_path
        if os.path.exists(mctal_file):
            mc = mctal.load(mctal_file)
            tally_nums = []
            del mc['metadata'] # don't need this
            for tal in mc:
                tally_nums.append(mc[tal].num)
                name = str(mc[tal].num)
                val = float(mc[tal].tfc.data[-1:][0][1])
                err = float(mc[tal].tfc.data[-1:][0][2])
                fom = float(mc[tal].tfc.data[-1:][0][3])
                run[name] = ([mc[tal].num,val,err,fom])
        
mctal_noww = cad_base_path+"no-ww"+mctal_path
mc = mctal.load(mctal_noww)
del mc['metadata'] # don't need this
for tal in mc:
    n = str(mc[tal].num)
    val = float(mc[tal].tfc.data[-1:][0][1])
    err = float(mc[tal].tfc.data[-1:][0][2])
    fom = float(mc[tal].tfc.data[-1:][0][3])
    results.append({"type": "no-ww", 
                    n:[mc[tal].num,val,err,fom]})

qr_tally = sorted([run for run in results \
                   if run["type"] == "qr"], \
                   key=lambda k: k['angles'])
ldo_tally = sorted([run for run in results \
                    if run["type"] == "ldo"], \
                    key=lambda k: k['angles'])
ldo_tally.pop(1) # take out LDO 05 run since it didn't finish
gkn_tally = sorted([run for run in results \
                    if run["type"] == "galerkin"], \
                    key=lambda k: k['angles'])[1:2] # only 04
ldfe_tally = sorted([run for run in results \
                     if run["type"] == "ldfe"], \
                     key=lambda k: k['angles'])[:2] # only 01 and 02

qa = []
for qt in qr_tally:
    qa.append(qt["angles"])
la = []
for lt in ldo_tally:
    la.append(lt["angles"])
ga = []
for gt in gkn_tally:
    ga.append(gt["angles"])
da = []
for dt in ldfe_tally:
    da.append(dt["angles"])
    
for t in tally_nums:
    print "Detector Tally #"+str(t)
    qm, qe, qo = np.array([]),np.array([]),np.array([])
    for qt in qr_tally:
        qm = np.append(qm,qt[str(t)][1]) # tally value
        qe = np.append(qe,qt[str(t)][2]) # tally rel. err.
        qo = np.append(qo,qt[str(t)][3]) # FoM
    gm, ge, go = np.array([]),np.array([]),np.array([])
    for gt in gkn_tally:
        gm = np.append(gm,gt[str(t)][1])
        ge = np.append(ge,gt[str(t)][2])
        go = np.append(go,gt[str(t)][3])
    lm, le, lo = np.array([]),np.array([]),np.array([])
    for lt in ldo_tally:
        lm = np.append(lm,lt[str(t)][1])
        le = np.append(le,lt[str(t)][2])
        lo = np.append(lo,lt[str(t)][3])
    dm, de, do = np.array([]),np.array([]),np.array([])
    for dt in ldfe_tally:
        dm = np.append(dm,dt[str(t)][1])
        de = np.append(de,dt[str(t)][2])
        do = np.append(do,dt[str(t)][3])
    print "QR   FOM " + str(qr_dict[str(t)][3])
    print "Gkn  FOM " + str(gkn_dict[str(t)][3])
    print "LDFE FOM " + str(ldfe_dict[str(t)][3])
    print "LDO  FOM " + str(ldo_dict[str(t)][3])
    print "avg  FOM " + str(np.mean([qr_dict[str(t)][3], \
                                   gkn_dict[str(t)][3], \
                                   ldfe_dict[str(t)][3], \
                                   ldo_dict[str(t)][3]]))
    print "QR   RE " + str(qr_dict[str(t)][2])
    print "Gkn  RE " + str(gkn_dict[str(t)][2])
    print "LDFE RE " + str(ldfe_dict[str(t)][2])
    print "LDO  RE " + str(ldo_dict[str(t)][2])
    print "avg  RE " + str(np.mean([qr_dict[str(t)][2], \
                                   gkn_dict[str(t)][2], \
                                   ldfe_dict[str(t)][2], \
                                   ldo_dict[str(t)][2]]))
    for run in results:
        if run["type"] == "no-ww" and str(t) in run:
            nw_tally = run[str(t)][1]
            nw_error = run[str(t)][2]
            nw_fom   = run[str(t)][3]
            print "analog FOM " + str(nw_fom)
            print "analog RE  " + str(nw_error)
            
for t in tally_nums:
    plt.figure()
    plt.subplot(1, 2, 1)
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    ax.ticklabel_format(style='sci',axis='y', scilimits=(0,0))
    plt.sca(ax)
    msize = 4.5
    plt.errorbar(qa, qm, fmt='o-', yerr=qm*qe, label = 'QR', mec='none',markersize=msize)
    plt.errorbar(ga, gm, fmt='s-', yerr=gm*ge, label = 'Galerkin', mec='none',markersize=msize)
    plt.errorbar(da, dm, fmt='^-', yerr=dm*de, label = 'LDFE', mec='none',markersize=msize)
    plt.errorbar(la, lm, fmt='D-', yerr=lm*le, label = 'LDO', mec='none',markersize=msize)
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Flux Tally [n/cm$^2$/s]")
    #
    plt.subplot(1, 2, 2)
    ax = plt.gca()
    colors = plt.cm.viridis(np.linspace(0,0.9,4))
    ax.set_prop_cycle(cycler('color', colors))
    plt.sca(ax)
    plt.plot(qa, qo, 'o-', label = 'QR', mec='none',markersize=msize)
    plt.plot(ga, go, 's-', label = 'Galerkin', mec='none',markersize=msize)
    plt.plot(da, do, '^-', label = 'LDFE', mec='none',markersize=msize)
    plt.plot(la, lo, 'D-', label = 'LDO', mec='none',markersize=msize)
    plt.xlabel('Number of Quadrature Points')
    plt.ylabel("Reported FOM")
    plt.yscale('log')
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    plt.legend(loc='lower right',fontsize='medium')
    fig = plt.gcf()
    fig.suptitle("Steel Plate Flux Tallies and FOM Values at Detector")
    plot_name = 'steel-cadis-'+str(t)+'.eps'
    plt.savefig(plot_dir+plot_name,format='eps')
    plt.close(plt.gcf())

Loading MCNP cell tallies...
Loading mctal file from steel-plate-cadis/galerkin/gkn04/output/mctal
INFO: Loading mctal from problem name 'steel-plate'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from steel-plate-cadis/galerkin/gkn08/output/mctal
INFO: Loading mctal from problem name 'steel-plate'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from steel-plate-cadis/galerkin/gkn06/output/mctal
INFO: Loading mctal from problem name 'steel-plate'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from steel-plate-cadis/ldfe/ldfe01/output/mctal
INFO: Loading mctal from problem name 'steel-plate'
            ...finished loading MCNP cell tallies
Loading MCNP cell tallies...
Loading mctal file from steel-plate-cadis/ldfe/ldfe02/output/mctal
INFO: Loading mctal from problem name 'steel-plate'
            ...finished loading MCNP cell tallies
L

Detector Tally #4
QR   FOM 0.05023
Gkn  FOM 0.0131723
LDFE FOM 2.92523
LDO  FOM 0.041564
avg  FOM 0.757549075
QR   RE 0.119165
Gkn  RE 0.0620312
LDFE RE 0.0149268
LDO  RE 0.126457
avg  RE 0.080645
analog FOM 0.00440725
analog RE  0.281038
analog FOM 0.00440725
analog RE  0.281038
analog FOM 0.00440725
analog RE  0.281038


MCNP FWCADIS

In [9]:
for run in results:
    if run["type"] != "no-ww":
        meshtal_file = fwc_base_path+run["type"]+"/"+run["name"]+meshtal_path
        if os.path.exists(meshtal_file):
            outp_file = fwc_base_path+run["type"]+"/"+run["name"]+outp_path
            f = open(outp_file)
            for line in f.readlines():
                if "computer time =" in line:
                    time = line.split()[-2]
                    if not time[0].isdigit():
                        time = float(time[1:])
                    else:
                        time = float(time)
                    units = line.split()[-1]
                    break
            f.close()
            run["time"] = time
            mc = meshtal.load(meshtal_file)['24:n'].extract()
            axes = mc.total.axes[:-1]
            tally_data = mc.data['total'].data[:,:,:,0]
            re_data    = mc.data['total'].data[:,:,:,1]
            tally_total = np.sum(tally_data)
            err = tally_data*re_data
            error_total = np.sqrt(np.sum(err*err))
            re_avg = np.mean(re_data)
            avg_fom = 1.0 / (re_avg*re_avg*time)
            run["fwc_tally"] = tally_total
            run["fwc_error"] = error_total
            run["fwc_fom"] = avg_fom

meshtal_noww = cad_base_path+"no-ww"+meshtal_path
mc = meshtal.load(meshtal_noww)['24:n'].extract()
axes = mc.total.axes[:-1]
tally_data = mc.data['total'].data[:,:,:,0]
re_data    = mc.data['total'].data[:,:,:,1]
tally_total = np.sum(tally_data)
err = tally_data*re_data
error_total = np.sqrt(np.sum(err*err))
re_avg = np.mean(re_data)
avg_fom = 1.0 / (re_avg*re_avg*time)
results.append({"type": "no-ww","fwc_tally":tally_total})
results.append({"type": "no-ww","fwc_error":error_total})
results.append({"type": "no-ww","fwc_fom":avg_fom})

# redoing the LDO tally since all LDO FWCADIS steel runs finished
# Galerkin 02 run didn't finish in either case so it can stay as-is
ldo_tally = sorted([run for run in results \
                    if run["type"] == "ldo"], \
                    key=lambda k: k['angles']) 

qm, qe, qo = np.array([]),np.array([]),np.array([])
for qt in qr_tally:
    qm = np.append(qm,qt["fwc_tally"]) # tally value
    qe = np.append(qe,qt["fwc_error"]) # tally err.
    qo = np.append(qo,qt["fwc_fom"])
gm, ge, go = np.array([]),np.array([]),np.array([])
for gt in gkn_tally:
    gm = np.append(gm,gt["fwc_tally"])
    ge = np.append(ge,gt["fwc_error"])
    go = np.append(go,gt["fwc_fom"])
la = []
lm, le, lo = np.array([]),np.array([]),np.array([])
for lt in ldo_tally:
    la.append(lt["angles"])
    lm = np.append(lm,lt["fwc_tally"])
    le = np.append(le,lt["fwc_error"])
    lo = np.append(lo,lt["fwc_fom"])
dm, de, do = np.array([]),np.array([]),np.array([])
for dt in ldfe_tally:
    dm = np.append(dm,dt["fwc_tally"])
    de = np.append(de,dt["fwc_error"])
    do = np.append(do,dt["fwc_fom"])
for run in results:
    if run["type"] == "no-ww" and run.has_key("fwc_tally"):
        nw_tally = run["fwc_tally"]
    if run["type"] == "no-ww" and run.has_key("fwc_error"):
        nw_error = run["fwc_error"]
    if run["type"] == "no-ww" and run.has_key("fwc_fom"):
        nw_fom = run["fwc_fom"]
    

plt.figure()
plt.subplot(1, 2, 1)
ax = plt.gca()
colors = plt.cm.viridis(np.linspace(0,0.9,4))
ax.set_prop_cycle(cycler('color', colors))
ax.ticklabel_format(style='sci',axis='y', scilimits=(0,0))
plt.sca(ax)
msize = 4.5
plt.errorbar(qa, qm, fmt='o-', yerr=qm*qe, label = 'QR', mec='none',markersize=msize)
plt.errorbar(ga, gm, fmt='s-', yerr=gm*ge, label = 'Galerkin', mec='none',markersize=msize)
plt.errorbar(da, dm, fmt='^-', yerr=dm*de, label = 'LDFE', mec='none',markersize=msize)
plt.errorbar(la, lm, fmt='D-', yerr=lm*le, label = 'LDO', mec='none',markersize=msize)
plt.xlabel('Number of Quadrature Points')
plt.ylabel("Flux Tally [n/cm$^2$/s]")
#
plt.subplot(1, 2, 2)
ax = plt.gca()
colors = plt.cm.viridis(np.linspace(0,0.9,4))
ax.set_prop_cycle(cycler('color', colors))
plt.sca(ax)
plt.plot(qa, qo, 'o-', label = 'QR', mec='none',markersize=msize)
plt.plot(ga, go, 's-', label = 'Galerkin', mec='none',markersize=msize)
plt.plot(da, do, '^-', label = 'LDFE', mec='none',markersize=msize)
plt.plot(la, lo, 'D-', label = 'LDO', mec='none',markersize=msize)
plt.xlabel('Number of Quadrature Points')
plt.ylabel("Average FOM")
plt.yscale('log')
ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
plt.legend(loc='lower right')
fig = plt.gcf()
fig.suptitle("Steel Plate Mesh Tally and Average FOM Values")
plot_name = 'steel-fwcadis.eps'
plt.savefig(plot_dir+plot_name,format='eps')
plt.close(plt.gcf())

Loading MCNP mesh tallies...
Loading meshtal file from steel-plate-fwcadis/galerkin/gkn04/output/meshtal
INFO: Problem name: steel-plate
            ...finished loading MCNP mesh tallies
Loading MCNP mesh tallies...
Loading meshtal file from steel-plate-fwcadis/galerkin/gkn08/output/meshtal
INFO: Problem name: steel-plate
            ...finished loading MCNP mesh tallies
Loading MCNP mesh tallies...
Loading meshtal file from steel-plate-fwcadis/galerkin/gkn06/output/meshtal
INFO: Problem name: steel-plate
            ...finished loading MCNP mesh tallies
Loading MCNP mesh tallies...
Loading meshtal file from steel-plate-fwcadis/ldfe/ldfe01/output/meshtal
INFO: Problem name: steel-plate
            ...finished loading MCNP mesh tallies
Loading MCNP mesh tallies...
Loading meshtal file from steel-plate-fwcadis/ldfe/ldfe02/output/meshtal
INFO: Problem name: steel-plate
            ...finished loading MCNP mesh tallies
Loading MCNP mesh tallies...
Loading meshtal file from steel-plate-fwca